internal covariant shift(ICS)：<br/>
深层神经网络在训练时会出现ICS现象：随着训练的进行，神经网络每层的参数会发生变化，进而会导致每层的输出的分布发生变化，这叫做ICS<br/>
ICS会带来两个问题：<br/>
1)高层的网络需要不断适应底层网络输出的分布变化，造成训练缓慢<br/>
2)不断变化的激活值的分布变化容易使训练进入饱和激活函数的饱和区，造成梯度消失，训练迟滞

解决ICS问题以前使用白化的方式，但会带来两个问题：<br/>
1)计算量较大<br/>
2)白化会抹除底层学习到的特征，使得底层的特征提取失效<br/>
针对以上两个问题，出现了batch_normalization的方法

batch-normalization的思想：<br/>
1)在每一个mini-batch对每个特征（每一个神经元）进行标准化，简化normlization计算过程<br/>
2)采用线性变换在一定程度上恢复底层特征的特征空间

具体操作：<br/>
$\mu_j = \frac{1}{m}\sum_{i=1}^{m}{z_j^{(i)}}$<br/>
$\delta^2_j = \frac{1}{m}\sum_{i=1}^{m}{(z_j^{(i)} - \mu_j)^2}$<br/>
$\hat z_j = \frac{z_j - \mu_j}{\sqrt{\delta_j^2 + \epsilon}}$<br/>
$其中，m为mini-batch的批大小，j表示某层的第j个神经元，z_j表示线性计算结果（wx+b），\epsilon为了防止分母为0$<br/>
$\pmb{注意：}batch norm是对每一层中的每个维度（每个神经元）单独计算均值与方差$

线性变换：<br/>
$\tilde z_j = \lambda_j \hat z_j + \beta_j$<br/>
$\pmb{注意：}一层中的每个神经元都有自己的\lambda_j与\beta_j参数$

batch norm在训练与测试时的不同：<br/>
训练时是对每一个mini-batch单独计算每一层的期望与方差，不能代表样本整体（因此也说batch norm带来了正则化效果）<br/>
测试时由于每次输入的样本很少，可能只有一个，因此计算期望与方差没有意义<br/>
一种标准的做法是模型训练好后，进行一遍推理，保留每个mini-batch的期望与方差，用它们得到整个数据集期望与方差的无偏估计，然后用作测试时的期望与方差：<br/>
$\mu_{test} = E(\mu_{batch})$<br/>
$\delta^2_{test} = \frac{m}{m-1}E(\delta^2_{batch})$<br/>

但在实际中，由于mini-batch的数量可能是百万级，比较占用空间，因此实际中经常采用指数滑动平均的方法来得测试阶段的期望与方差：<br/>
$\hat \mu_k = \beta \hat \mu_{k-1} + (1- \beta)\mu_k$<br/>
$\hat \delta^2_k = \beta \hat \delta^2_{k-1} + (1- \beta)\delta^2_k$<br/>
$其中，k为第k个mini-batch$

BN的好处：<br/>
1)缓解ICS现象，使得每层的输入分布较稳定，各层之间解耦合，高层网络无需适应底层输入的变化，加速网络训练<br/>
2)由于每层输出固定在一定范围，对于饱和激活函数，可以缓解梯度消失现象<br/>
3)使得网络对权重初始化、学习率不那么敏感，无需小心地设置权值初始化方法，学习率也可以相对较大<br/>
4)batch norm是针对每一个mini-batch进行normalization操作，带来了一定的噪声，加入了正则化效果（甚至可以代替dropout）